To get started you need to install all the necessary project dependencies. You can do it by running the cell below. Exclamation mark means that terminal command shall be executed in the cell

In [1]:
! pip install -r requirements.txt

  Using cached matplotlib-3.6.1-cp39-cp39-win_amd64.whl (7.2 MB)
  Using cached numpy-1.21.0-cp39-cp39-win_amd64.whl (14.0 MB)

ERROR: Could not install packages due to an OSError: [WinError 5] Access is denied: 'C:\\Users\\andre\\anaconda3\\Lib\\site-packages\\numpy\\.libs\\libopenblas.GK7GX5KEQ4F6UYO3P26ULGBQYHGQO7J4.gfortran-win_amd64.dll'
Consider using the `--user` option or check the permissions.




  Attempting uninstall: numpy
    Found existing installation: numpy 1.21.5
    Uninstalling numpy-1.21.5:
      Successfully uninstalled numpy-1.21.5
  Rolling back uninstall of numpy
  Moving to c:\users\andre\anaconda3\lib\site-packages\numpy-1.21.5.dist-info\
   from C:\Users\andre\anaconda3\Lib\site-packages\~umpy-1.21.5.dist-info
  Moving to c:\users\andre\anaconda3\lib\site-packages\numpy\__config__.py
   from C:\Users\andre\AppData\Local\Temp\pip-uninstall-nfdy0s9v\__config__.py
  Moving to c:\users\andre\anaconda3\lib\site-packages\numpy\__init__.cython-30.pxd
   from C:\Users\andre\AppData\Local\Temp\pip-uninstall-nfdy0s9v\__init__.cython-30.pxd
  Moving to c:\users\andre\anaconda3\lib\site-packages\numpy\__init__.pxd
   from C:\Users\andre\AppData\Local\Temp\pip-uninstall-nfdy0s9v\__init__.pxd
  Moving to c:\users\andre\anaconda3\lib\site-packages\numpy\__init__.py
   from C:\Users\andre\AppData\Local\Temp\pip-uninstall-nfdy0s9v\__init__.py
  Moving to c:\users\andre\anacon

In [2]:
#pip install optuna

  Moving to c:\users\andre\anaconda3\lib\site-packages\numpy\core\tests\test_mem_overlap.py
   from C:\Users\andre\AppData\Local\Temp\pip-uninstall-nfdy0s9v\core\tests\test_mem_overlap.py
  Moving to c:\users\andre\anaconda3\lib\site-packages\numpy\core\tests\test_memmap.py
   from C:\Users\andre\AppData\Local\Temp\pip-uninstall-nfdy0s9v\core\tests\test_memmap.py
  Moving to c:\users\andre\anaconda3\lib\site-packages\numpy\core\tests\test_multiarray.py
   from C:\Users\andre\AppData\Local\Temp\pip-uninstall-nfdy0s9v\core\tests\test_multiarray.py
  Moving to c:\users\andre\anaconda3\lib\site-packages\numpy\core\tests\test_nditer.py
   from C:\Users\andre\AppData\Local\Temp\pip-uninstall-nfdy0s9v\core\tests\test_nditer.py
  Moving to c:\users\andre\anaconda3\lib\site-packages\numpy\core\tests\test_numeric.py
   from C:\Users\andre\AppData\Local\Temp\pip-uninstall-nfdy0s9v\core\tests\test_numeric.py
  Moving to c:\users\andre\anaconda3\lib\site-packages\numpy\core\tests\test_numerictypes.

Here we import necessary packages. Note that utils is a folder, not a package. In this folder we need "machine_learning.py" file, from which we import MachineLearning object. We also suppress warnings, since they make the output messy

In [1]:
import pickle, os, warnings, optuna, numpy as np
from utils.machine_learning import MachineLearning
warnings.filterwarnings('ignore')

/Users/grigoryturchenko/tensorflow-test/env/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Here we cycle through the files and load only training and test datasets

In [2]:
data_sets = []
directory = 'data'

files = os.listdir(directory)
files.sort()
files.pop(files.index('AB_NYC_2019.csv'))
files.pop(files.index('X_cluster.pkl'))
for file in files:
    with open(f"{directory}/{file}", 'rb') as f:
        data_sets.append(pickle.load(f))
X_test, X_train, y_test, y_train = data_sets[0], data_sets[1], data_sets[2], data_sets[3]

Setting random seed for reproducibility and creating an instance of MachineLearning class. As an input it accepts one of:
- [linreg](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html)
    - No sensible hyperparameters to tune
- [knn](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsRegressor.html)
    - **Andreas:** Start with ('n_neighbors', 1, 10), ('weights', ['uniform', 'distance'])
- [tree](https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeRegressor.html)
    - **Waqar:** Start with ('max_depth', 3, 12), ('min_samples_split', 10, 20), ('min_samples_leaf', 1, 10), ('max_features', ['auto', 'sqrt'])
- [rf](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestRegressor.html)
- [xgb](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.GradientBoostingRegressor.html#sklearn.ensemble.GradientBoostingRegressor)

You can click an algorithm and check out what kinds of parameteres it has for tuning

In [9]:
np.random.seed(510)
ml = MachineLearning('knn')

This is the most important part of the notebook, since here we tune the hyperparameters and train the model. You should supply number of hyperparameters tuning iterations, training and test datasets (in this particular order), and hyperparameters for tuning (in any order). A few notes about hyperparameters:
- They can be either numeric or categorical
- If you want to supply a numeric (integer or float) parameter, you pass the following tuple: (hyperparameter_name, min, max)
- If you want to supply a categorical parameter, you pass the following tuple: (hyperparameter_name, [val_1, val_2, ..., val_n])
- You may supply as many hyperparameters you want, from 1 to as many as an algo has

Sometimes the training might take quite a while if you pass many hyperparameters with wide values range. However, the algorithm does not really try all possible combinations of hyperparameters, as it gets rid of not promising combinations quite early, so do not be afraid of experimenting!

In [6]:
ml.fit(100, X_train, y_train, X_test, y_test, ('n_neighbors',1 , 10), ('weights', ['uniform', 'distance']))

After the training is done, we can check out the hyperparameters the model was trained with

In [7]:
ml.model.get_params()

{'algorithm': 'auto',
 'leaf_size': 30,
 'metric': 'minkowski',
 'metric_params': None,
 'n_jobs': None,
 'n_neighbors': 10,
 'p': 2,
 'weights': 'uniform'}

In [8]:
pred_train = ml.predict(X_train)
pred_test = ml.predict(X_test)
print(f'Train RMSE: {ml.get_rmse(y_train, pred_train)}\nTest RMSE:  {ml.get_rmse(y_test, pred_test)}')

Train RMSE: 0.4266588306362564
Test RMSE:  0.48170423887938885


In [9]:
ml = MachineLearning('knn')

In [10]:
ml.fit(100, X_train, y_train, X_test, y_test, ('n_neighbors',1 , 11), ('weights', ['uniform', 'distance']))

In [11]:
ml.model.get_params()

{'algorithm': 'auto',
 'leaf_size': 30,
 'metric': 'minkowski',
 'metric_params': None,
 'n_jobs': None,
 'n_neighbors': 11,
 'p': 2,
 'weights': 'uniform'}

In [12]:
pred_train = ml.predict(X_train)
pred_test = ml.predict(X_test)
print(f'Train RMSE: {ml.get_rmse(y_train, pred_train)}\nTest RMSE:  {ml.get_rmse(y_test, pred_test)}')

Train RMSE: 0.4299462499244671
Test RMSE:  0.4806614992234741


You shouldn't stop at this point, as our goal is to try and improve the performance as much as we can. For this case, you can see that `n_neighbors` parameter is equal to 10, which is the upper limit of what we supplied to the `fit` method. That means that we can try a higher range of neighbors and see if it makes RMSE even lower. Let us create an instance of the class once again and fit the model with different params

Check out the hyperparameters

Check out the metrics. Now as we increased the number of neighbors the model is overfitted, because Train RMSE is 0 (the model perfectly predicted all the training examples), but Test RMSE is not much less. The model once again used the upper limit of `n_neighbors`, meaning that increasing this number further does not make much sense, and we can stick to the previous version

In [11]:
np.random.seed(510)
ml = MachineLearning('knn')

In [12]:
ml.fit(100, X_train, y_train, X_test, y_test, ('n_neighbors',1 , 12), ('weights', ['uniform', 'distance']))

In [13]:
ml.model.get_params()

{'algorithm': 'auto',
 'leaf_size': 30,
 'metric': 'minkowski',
 'metric_params': None,
 'n_jobs': None,
 'n_neighbors': 12,
 'p': 2,
 'weights': 'distance'}

In [14]:
pred_train = ml.predict(X_train)
pred_test = ml.predict(X_test)
print(f'Train RMSE: {ml.get_rmse(y_train, pred_train)}\nTest RMSE:  {ml.get_rmse(y_test, pred_test)}')

Train RMSE: 0.0
Test RMSE:  0.4721615142547624


In [7]:
np.random.seed(510)
ml = MachineLearning('knn')

In [8]:
ml.fit(100, X_train, y_train, X_test, y_test, ('n_neighbors',1 ,13), ('weights', ['uniform', 'distance']))

In [9]:
ml.model.get_params()

{'algorithm': 'auto',
 'leaf_size': 30,
 'metric': 'minkowski',
 'metric_params': None,
 'n_jobs': None,
 'n_neighbors': 13,
 'p': 2,
 'weights': 'distance'}

In [10]:
pred_train = ml.predict(X_train)
pred_test = ml.predict(X_test)
print(f'Train RMSE: {ml.get_rmse(y_train, pred_train)}\nTest RMSE:  {ml.get_rmse(y_test, pred_test)}')

Train RMSE: 0.0
Test RMSE:  0.471298494524852


In [3]:
np.random.seed(510)
ml = MachineLearning('knn')

In [4]:
ml.fit(100, X_train, y_train, X_test, y_test, ('n_neighbors',1 , 14), ('weights', ['uniform', 'distance']))

In [5]:
ml.model.get_params()

{'algorithm': 'auto',
 'leaf_size': 30,
 'metric': 'minkowski',
 'metric_params': None,
 'n_jobs': None,
 'n_neighbors': 14,
 'p': 2,
 'weights': 'distance'}

In [6]:
pred_train = ml.predict(X_train)
pred_test = ml.predict(X_test)
print(f'Train RMSE: {ml.get_rmse(y_train, pred_train)}\nTest RMSE:  {ml.get_rmse(y_test, pred_test)}')

Train RMSE: 0.0
Test RMSE:  0.4706209694095188


In [22]:
ml = MachineLearning('knn')

In [23]:
ml.fit(100, X_train, y_train, X_test, y_test, ('n_neighbors',1 , 15), ('weights', ['uniform', 'distance']))

In [24]:
ml.model.get_params()

{'algorithm': 'auto',
 'leaf_size': 30,
 'metric': 'minkowski',
 'metric_params': None,
 'n_jobs': None,
 'n_neighbors': 15,
 'p': 2,
 'weights': 'distance'}

In [25]:
pred_train = ml.predict(X_train)
pred_test = ml.predict(X_test)
print(f'Train RMSE: {ml.get_rmse(y_train, pred_train)}\nTest RMSE:  {ml.get_rmse(y_test, pred_test)}')

Train RMSE: 0.0
Test RMSE:  0.47032948566061383


In [15]:
np.random.seed(510)
ml = MachineLearning('knn')

In [16]:
ml.fit(100, X_train, y_train, X_test, y_test, ('n_neighbors',1 , 16), ('weights', ['uniform', 'distance']))

In [17]:
ml.model.get_params()

{'algorithm': 'auto',
 'leaf_size': 30,
 'metric': 'minkowski',
 'metric_params': None,
 'n_jobs': None,
 'n_neighbors': 16,
 'p': 2,
 'weights': 'distance'}

In [18]:
pred_train = ml.predict(X_train)
pred_test = ml.predict(X_test)
print(f'Train RMSE: {ml.get_rmse(y_train, pred_train)}\nTest RMSE:  {ml.get_rmse(y_test, pred_test)}')

Train RMSE: 0.0
Test RMSE:  0.46951913055551714


In [19]:
ml.save()

In [33]:
ml = MachineLearning('knn')

In [34]:
ml.fit(100, X_train, y_train, X_test, y_test, ('n_neighbors',1 , 17), ('weights', ['uniform', 'distance']))

In [35]:
ml.model.get_params()

{'algorithm': 'auto',
 'leaf_size': 30,
 'metric': 'minkowski',
 'metric_params': None,
 'n_jobs': None,
 'n_neighbors': 17,
 'p': 2,
 'weights': 'distance'}

In [36]:
pred_train = ml.predict(X_train)
pred_test = ml.predict(X_test)
print(f'Train RMSE: {ml.get_rmse(y_train, pred_train)}\nTest RMSE:  {ml.get_rmse(y_test, pred_test)}')

Train RMSE: 0.0
Test RMSE:  0.47609192326060523


In [14]:
np.random.seed(510)
ml = MachineLearning('knn')

In [15]:
ml.fit(100, X_train, y_train, X_test, y_test, ('n_neighbors',1 , 18), ('weights', ['uniform', 'distance']))

In [16]:
ml.model.get_params()

{'algorithm': 'auto',
 'leaf_size': 30,
 'metric': 'minkowski',
 'metric_params': None,
 'n_jobs': None,
 'n_neighbors': 18,
 'p': 2,
 'weights': 'distance'}

In [17]:
pred_train = ml.predict(X_train)
pred_test = ml.predict(X_test)
print(f'Train RMSE: {ml.get_rmse(y_train, pred_train)}\nTest RMSE:  {ml.get_rmse(y_test, pred_test)}')

Train RMSE: 0.0
Test RMSE:  0.4686815616466524


In [41]:
ml = MachineLearning('knn')

In [42]:
ml.fit(100, X_train, y_train, X_test, y_test, ('n_neighbors',1 , 19), ('weights', ['uniform', 'distance']))

In [43]:
ml.model.get_params()

{'algorithm': 'auto',
 'leaf_size': 30,
 'metric': 'minkowski',
 'metric_params': None,
 'n_jobs': None,
 'n_neighbors': 19,
 'p': 2,
 'weights': 'distance'}

In [44]:
pred_train = ml.predict(X_train)
pred_test = ml.predict(X_test)
print(f'Train RMSE: {ml.get_rmse(y_train, pred_train)}\nTest RMSE:  {ml.get_rmse(y_test, pred_test)}')

Train RMSE: 0.0
Test RMSE:  0.4746736506339132


In [45]:
ml = MachineLearning('knn')

In [46]:
ml.fit(100, X_train, y_train, X_test, y_test, ('n_neighbors',1 , 20), ('weights', ['uniform', 'distance']))

In [47]:
ml.model.get_params()

{'algorithm': 'auto',
 'leaf_size': 30,
 'metric': 'minkowski',
 'metric_params': None,
 'n_jobs': None,
 'n_neighbors': 20,
 'p': 2,
 'weights': 'distance'}

In [48]:
pred_train = ml.predict(X_train)
pred_test = ml.predict(X_test)
print(f'Train RMSE: {ml.get_rmse(y_train, pred_train)}\nTest RMSE:  {ml.get_rmse(y_test, pred_test)}')

Train RMSE: 0.0
Test RMSE:  0.47410731398884537


In [49]:
#We choose the below model as the most appropriete (lowest train rmse)

In [50]:
ml = MachineLearning('knn')
ml.fit(100, X_train, y_train, X_test, y_test, ('n_neighbors', 1, 16), ('weights', ['uniform', 'distance']))

Once we made up our mind with the model, we should save it by running this cell. Make sure you are saving the right model! I reran the first model in the cell above to save this particular version of KNN regressor

In [51]:
ml.save()

If you still have some issues with how the code works, you can use the help function to read the documentation

In [52]:
help(ml.fit)

Help on method fit in module utils.machine_learning:

fit(n_trials: int, X_train: numpy.ndarray, y_train: numpy.ndarray, X_test: numpy.ndarray, y_test: numpy.ndarray, *args) -> None method of utils.machine_learning.MachineLearning instance
    Find optimal hyperparameters and fit the model
    
    ## Parameters
    `n_trials` : int
        Number of optimization iterations
    
    `X_train`, `y_train`, `X_test`, `y_test` : np.ndarray
        Training and test datasets
    
    `*args` : tuple
        Tuples containing hyperparameters and their values. If a hyperparameter is:
        - Numeric, then the tuple passed is (hyperparameter_name, min, max)
        - Categorical, then the tuple passed is (hyperparameter_name, [val_1, val_2, ..., val_n])

